In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf

In [2]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
def create_data(df):
    
    df['gill-attachment'] = df['gill-attachment'].fillna('z')
    df['ring-type'] = df['ring-type'].fillna('a')
    df['season'] = df['season'].fillna('z')
    
    data = pd.DataFrame()
    
    
    #---CAT---#
    data['c0'] = df['cap-shape'].map({'x':0, 'f':1, 's':2, 'b':3, 'o':4, 'p':5, 'c':6})
    
    data['c1'] = df['cap-color'].map({'n':0, 'y':1, 'w':2, 'g':3, 'e':4, 'o':5, 'r':6,
                                      'u':7, 'p':8, 'k':9, 'b':10, 'l':11})
    
    data['c2'] = df['gill-attachment'].map({'a':0, 'd':1, 'x':2, 'p':3, 's':4, 'e':5, 'f':6, 'z':7})
    
    data['c3'] = df['gill-color'].map({'w':0, 'n':1, 'y':2, 'p':3, 'g':4, 'f':5, 'o':6,
                                       'k':7, 'r':8, 'e':9, 'b':10, 'u':11})
    
    data['c4'] = df['stem-color'].map({'w':0, 'n':1, 'y':2, 'g':3, 'o':4, 'e':5, 'u':6,
                                       'f':7, 'p':8, 'k':9, 'r':10, 'l':11, 'b':12})
    
    data['c5'] = df['ring-type'].map({'f':0, 'e':1, 'z':2, 'l':3, 'r':4, 'p':5, 'g':6, 'm':7, 'a':8})
    
    data['c6'] = df['habitat'].map({'d':0, 'g':1, 'l':2, 'm':3, 'h':4, 'p':5, 'w':6, 'u':7})
    
    #---BIN---#
    data['b0'] = df['edible-poisonous'].map({'p':1, 'e':0})
    data['b1'] = df['does-bruise-or-bleed'].map({'t':1, 'f':0})
    data['b2'] = df['has-ring'].map({'t':1, 'f':0})
    
    
    #---NUM---#
    data['n0'] = (df['cap-diameter'] - 30) / 30.0
    data['n1'] = (df['stem-height'] - 10) / 20.0
    data['n2'] = (df['stem-width'] - 40) / 60.0
    data['n3'] = data['n0'] / data['n2']
    data['n4'] = data['n1'] / data['n2']
    
    data['n3'] = data['n3'].replace([np.inf, -np.inf], 0.0) / 400
    data['n4'] = data['n4'].replace([np.inf, -np.inf], 0.0) / 400
    
    data['y'] = df['season'].map({'a':0, 'u':1, 'w':2, 's':3, 'z':-1})
    
    return data.reset_index(drop=True)
    
    

data = create_data(data)
test = create_data(test) 

cols = [f'c{i}' for i in range(7)] + ['b0', 'b1', 'b2'] + ['n0', 'n1', 'n2', 'n3', 'n4']

data = pd.get_dummies(data, columns=[f'c{i}' for i in range(7)])
test = pd.get_dummies(test, columns=[f'c{i}' for i in range(7)])

cols = data.columns.to_list()
cols.remove('y')
cols

['b0',
 'b1',
 'b2',
 'n0',
 'n1',
 'n2',
 'n3',
 'n4',
 'c0_0',
 'c0_1',
 'c0_2',
 'c0_3',
 'c0_4',
 'c0_5',
 'c0_6',
 'c1_0',
 'c1_1',
 'c1_2',
 'c1_3',
 'c1_4',
 'c1_5',
 'c1_6',
 'c1_7',
 'c1_8',
 'c1_9',
 'c1_10',
 'c1_11',
 'c2_0',
 'c2_1',
 'c2_2',
 'c2_3',
 'c2_4',
 'c2_5',
 'c2_6',
 'c2_7',
 'c3_0',
 'c3_1',
 'c3_2',
 'c3_3',
 'c3_4',
 'c3_5',
 'c3_6',
 'c3_7',
 'c3_8',
 'c3_9',
 'c3_10',
 'c3_11',
 'c4_0',
 'c4_1',
 'c4_2',
 'c4_3',
 'c4_4',
 'c4_5',
 'c4_6',
 'c4_7',
 'c4_8',
 'c4_9',
 'c4_10',
 'c4_11',
 'c4_12',
 'c5_0',
 'c5_1',
 'c5_2',
 'c5_3',
 'c5_4',
 'c5_5',
 'c5_6',
 'c5_7',
 'c5_8',
 'c6_0',
 'c6_1',
 'c6_2',
 'c6_3',
 'c6_4',
 'c6_5',
 'c6_6',
 'c6_7']

In [4]:
N_folds = 5
seeds = [455, 485, 659, 16, 5659]

data_preds = np.zeros((data.shape[0], 4))
test_preds = np.zeros((test.shape[0], 4))

feature_importance = pd.DataFrame()

for seed in seeds:
    print('---------------------------------------')
    print('SEED : ', seed)
    print('---------------------------------------')
    
    
    data['fold'] = -1
    skf = StratifiedKFold(n_splits=N_folds, shuffle=True, random_state=seed)
    for f, (_, idxs) in enumerate(skf.split(data, data['y'])):
        data.loc[idxs, 'fold'] = f
        
    for F in range(N_folds):
        
        print('FOLD : ', F)
        
        train = data[data['fold'] != F].reset_index(drop=True)
        valid = data[data['fold'] == F].reset_index(drop=True)
        
        
        
        model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(len(cols))),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])
        
        model.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(train[cols], train['y'], 
                  validation_data=(valid[cols], valid['y']), 
                     epochs=25, batch_size=64, verbose=1)
        
        
        preds = model.predict(valid[cols])
        data_preds[data[data['fold'] == F].index] += preds
        
        preds = model.predict(test[cols])
        test_preds += preds

---------------------------------------
SEED :  455
---------------------------------------
FOLD :  0
Epoch 1/25
535/535 [==============================] - 1s 2ms/step - loss: 0.9816 - accuracy: 0.4964 - val_loss: 0.8821 - val_accuracy: 0.4926
Epoch 2/25
535/535 [==============================] - 1s 2ms/step - loss: 0.8561 - accuracy: 0.5113 - val_loss: 0.8068 - val_accuracy: 0.5076
Epoch 3/25
535/535 [==============================] - 1s 2ms/step - loss: 0.8057 - accuracy: 0.5162 - val_loss: 0.7678 - val_accuracy: 0.5161
Epoch 4/25
535/535 [==============================] - 1s 2ms/step - loss: 0.7826 - accuracy: 0.5192 - val_loss: 0.7548 - val_accuracy: 0.5137
Epoch 5/25
535/535 [==============================] - 1s 2ms/step - loss: 0.7656 - accuracy: 0.5227 - val_loss: 0.7460 - val_accuracy: 0.5159
Epoch 6/25
535/535 [==============================] - 1s 2ms/step - loss: 0.7567 - accuracy: 0.5300 - val_loss: 0.7427 - val_accuracy: 0.5048
Epoch 7/25
535/535 [==========================

KeyboardInterrupt: 

In [5]:
data_preds = data_preds / (len(seeds))
test_preds = test_preds / (len(seeds) * N_folds)

In [ ]:
sub = pd.DataFrame()
sub['season'] = np.argmax(test_preds, axis = 1)
sub['season'] = sub['season'].map({0:'a', 1:'u', 2:'w', 3:'s'})
sub.to_csv('tf.csv', index=False)
sub['season'].value_counts(normalize = True)

In [ ]:
np.save('tf1_oof.npy', data_preds)
np.save('tf1_preds.npy', test_preds)